In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'1nnr9qit'

In [3]:
wandb.init(id='1xe5sewu', project="carla", resume="allow")

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

14029.957

In [5]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [6]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [7]:
%%time
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

CPU times: user 56.2 s, sys: 36.9 s, total: 1min 33s
Wall time: 1min 33s


In [8]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs


In [9]:
"""for n, p in m.named_parameters():
    if 'fcs_2' in n:
        p.requires_grad = True
    else:
        p.requires_grad = False
        
    print(n, p.requires_grad)"""

"for n, p in m.named_parameters():\n    if 'fcs_2' in n:\n        p.requires_grad = True\n    else:\n        p.requires_grad = False\n        \n    print(n, p.requires_grad)"

In [10]:
model_stem = "8.22"
START_E = 34

In [11]:
n_epochs = 1000
log_wandb = True

for e in range(START_E, n_epochs):
    
    opt.param_groups[0]['lr'] = 3e-4 if e < 10 else 1e-4

    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=1280)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 34

{'trn_control_loss': 0.00918275, 'trn_headings_loss': 0.00044244, 'trn_curvatures_loss': 0.00171769, 'consistency losses/trn_steer_cost': 0.00407322, 'consistency losses/trn_te_loss': 21.91909285, 'aux losses/trn_pitch_loss': 0.07907557, 'aux losses/trn_yaw_loss': 0.14802861, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.546875, 'consistency losses/trn_torque_loss': 2651.10220098, 'consistency losses/trn_torque_delta_loss': 8.22909235, 'logistical/obs_consumed_per_second': 223.40944882, 'logistical/obs_generated_per_second': 105.05511811, 'logistical/data_consumption_ratio': 2.16301906, 'logistical/manual_train_pause': 7.874e-05, 'logistical/max_param': 16.16460991, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 231.4140625, 'logistical/obs_generated_per_second': 103.7109375, 'logistical/data_consumption_ratio': 2.23372875, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00682083, 'trn_headings_loss': 0.00031839, 'trn_curvatures_loss':

{'logistical/obs_consumed_per_second': 233.84375, 'logistical/obs_generated_per_second': 109.9765625, 'logistical/data_consumption_ratio': 2.12751415, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00599779, 'trn_headings_loss': 0.00031589, 'trn_curvatures_loss': 0.00110254, 'consistency losses/trn_steer_cost': 0.00262384, 'consistency losses/trn_te_loss': 22.18805114, 'aux losses/trn_pitch_loss': 0.06874108, 'aux losses/trn_yaw_loss': 0.09703636, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.40625, 'consistency losses/trn_torque_loss': 2625.27478743, 'consistency losses/trn_torque_delta_loss': 7.89324755, 'logistical/max_param': 16.16609955, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 234.9375, 'logistical/obs_generated_per_second': 105.375, 'logistical/data_consumption_ratio': 2.2325187, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00879678, 'trn_headings_loss': 0.0004612, 'trn_curvatures_loss': 0.00159169, 'consistency loss

{'logistical/obs_consumed_per_second': 234.1953125, 'logistical/obs_generated_per_second': 106.9453125, 'logistical/data_consumption_ratio': 2.19170724, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00687554, 'trn_headings_loss': 0.00034956, 'trn_curvatures_loss': 0.00110686, 'consistency losses/trn_steer_cost': 0.00271405, 'consistency losses/trn_te_loss': 22.75575735, 'aux losses/trn_pitch_loss': 0.07364225, 'aux losses/trn_yaw_loss': 0.09427404, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.5234375, 'consistency losses/trn_torque_loss': 2770.92666149, 'consistency losses/trn_torque_delta_loss': 7.47417637, 'logistical/max_param': 16.17186737, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 239.0703125, 'logistical/obs_generated_per_second': 99.40625, 'logistical/data_consumption_ratio': 2.40042504, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0062483, 'trn_headings_loss': 0.00036321, 'trn_curvatures_loss': 0.00140788, 'consist

{'logistical/obs_consumed_per_second': 221.8671875, 'logistical/obs_generated_per_second': 100.609375, 'logistical/data_consumption_ratio': 2.20894121, 'logistical/manual_train_pause': 0.00023438, 'trn_control_loss': 0.00704576, 'trn_headings_loss': 0.00041372, 'trn_curvatures_loss': 0.00149972, 'consistency losses/trn_steer_cost': 0.00368537, 'consistency losses/trn_te_loss': 21.2327206, 'aux losses/trn_pitch_loss': 0.07219577, 'aux losses/trn_yaw_loss': 0.1177237, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.4765625, 'consistency losses/trn_torque_loss': 2779.99970341, 'consistency losses/trn_torque_delta_loss': 7.88968478, 'logistical/max_param': 16.1733036, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 240.015625, 'logistical/obs_generated_per_second': 108.4140625, 'logistical/data_consumption_ratio': 2.21650597, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00871858, 'trn_headings_loss': 0.00045464, 'trn_curvatures_loss': 0.00180024, 'c

{'logistical/obs_consumed_per_second': 233.9609375, 'logistical/obs_generated_per_second': 100.3671875, 'logistical/data_consumption_ratio': 2.3314033, 'logistical/manual_train_pause': 7.812e-05, 'trn_control_loss': 0.00747265, 'trn_headings_loss': 0.00044357, 'trn_curvatures_loss': 0.00149216, 'consistency losses/trn_steer_cost': 0.00340199, 'consistency losses/trn_te_loss': 20.89388664, 'aux losses/trn_pitch_loss': 0.07626963, 'aux losses/trn_yaw_loss': 0.11032295, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.40625, 'consistency losses/trn_torque_loss': 2895.35231113, 'consistency losses/trn_torque_delta_loss': 7.5014003, 'logistical/max_param': 16.17404747, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 229.390625, 'logistical/obs_generated_per_second': 96.5234375, 'logistical/data_consumption_ratio': 2.37987123, 'logistical/manual_train_pause': 0.00070312, 'trn_control_loss': 0.00969779, 'trn_headings_loss': 0.00035878, 'trn_curvatures_loss': 0.0014072

...

{'logistical/obs_consumed_per_second': 234.5, 'logistical/obs_generated_per_second': 102.421875, 'logistical/data_consumption_ratio': 2.28896499, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00439283, 'trn_headings_loss': 0.00024316, 'trn_curvatures_loss': 0.00078139, 'consistency losses/trn_steer_cost': 0.00231371, 'consistency losses/trn_te_loss': 22.72098106, 'aux losses/trn_pitch_loss': 0.06241643, 'aux losses/trn_yaw_loss': 0.07743478, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.484375, 'consistency losses/trn_torque_loss': 3026.86466455, 'consistency losses/trn_torque_delta_loss': 6.01204746, 'logistical/max_param': 16.18266106, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 232.1640625, 'logistical/obs_generated_per_second': 96.4453125, 'logistical/data_consumption_ratio': 2.40693049, 'logistical/manual_train_pause': 0.00132812, 'trn_control_loss': 0.00611205, 'trn_headings_loss': 0.0002953, 'trn_curvatures_loss': 0.00133646, 'consis

{'logistical/obs_consumed_per_second': 235.1875, 'logistical/obs_generated_per_second': 99.640625, 'logistical/data_consumption_ratio': 2.36339785, 'logistical/manual_train_pause': 0.00070312, 'trn_control_loss': 0.00605924, 'trn_headings_loss': 0.00030029, 'trn_curvatures_loss': 0.0011718, 'consistency losses/trn_steer_cost': 0.00351509, 'consistency losses/trn_te_loss': 22.62849692, 'aux losses/trn_pitch_loss': 0.06386554, 'aux losses/trn_yaw_loss': 0.11046779, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.546875, 'consistency losses/trn_torque_loss': 2895.71094513, 'consistency losses/trn_torque_delta_loss': 7.13032584, 'logistical/max_param': 16.18482208, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 239.4296875, 'logistical/obs_generated_per_second': 98.109375, 'logistical/data_consumption_ratio': 2.44546624, 'logistical/manual_train_pause': 0.00210938, 'trn_control_loss': 0.00723062, 'trn_headings_loss': 0.00033606, 'trn_curvatures_loss': 0.00172411,

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_1564899/4029689652.py", line 10, in <module>
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler,
  File "/home/beans/bespoke/train_utils.py", line 296, in run_epoch
    scaler.scale(loss).backward()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/_tensor.py", line 307, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/autograd/__init__.py", line 154, in backward
    Variable._execution_engine.run_backward(
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/IPy

TypeError: object of type 'NoneType' has no len()

In [12]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e66.torch")

Error in callback <function _WandbInit._resume_backend at 0x7f090b5bf670> (for pre_run_cell):


Exception: The wandb backend process has shutdown

Error in callback <function _WandbInit._pause_backend at 0x7f090b5bf550> (for post_run_cell):


Exception: The wandb backend process has shutdown

Exception in thread Thread-11:
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 149, in check_network_status
    status_response = self._interface.communicate_network_status()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface.py", line 120, in communicate_network_status
    resp = self._communicate_network_status(status)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface_queue.py", line 411, in _communicate_network_status
    resp = self._communicate(req, local=True)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface

In [13]:
set_should_stop(True)

Error in callback <function _WandbInit._resume_backend at 0x7f090b5bf670> (for pre_run_cell):


Exception: The wandb backend process has shutdown

Error in callback <function _WandbInit._pause_backend at 0x7f090b5bf550> (for post_run_cell):


Exception: The wandb backend process has shutdown

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 167, in check_status
    status_response = self._interface.communicate_stop_status()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface.py", line 109, in communicate_stop_status
    resp = self._communicate_stop_status(status)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface_queue.py", line 401, in _communicate_stop_status
    resp = self._communicate(req, local=True)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface_queue.py", line 232

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
